## Rule Based Labelling - Awesome Oscillator

In [154]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from pylab import plt
import mplfinance as mpf
#from talib.Momentum import AwesomeOscillatorIndicator
import talib
from ta.momentum import AwesomeOscillatorIndicator
from zigzag import peak_valley_pivots
"""
for interactive plots (first - pip3 install ipywidgets)
1. pip3 install ipywidgets
for jupyter lab
2. jupyter labextension install @jupyter-widgets/jupyterlab-manager
or for jupyter notebook
2. !jupyter nbextension enable --py widgetsnbextension
"""

'\nfor interactive plots (first - pip3 install ipywidgets)\n1. pip3 install ipywidgets\nfor jupyter lab\n2. jupyter labextension install @jupyter-widgets/jupyterlab-manager\nor for jupyter notebook\n2. !jupyter nbextension enable --py widgetsnbextension\n'

In [155]:
df_EURUSDM30 = pd.read_csv('EURUSD_M30.csv')
with pd.option_context('display.precision', 20):  
    #dataM1 = pd.DataFrame(df_M1)

    dataM30=pd.DataFrame(df_EURUSDM30)
    # show all the columns
    pd.set_option('display.max_columns', None)
    # show all the rows
    pd.set_option('display.max_rows', None)
dataM30['Time'] = pd.to_datetime(dataM30['Time'])

In [156]:
print(dataM30.shape)
dataM30.tail()

(169288, 6)


,Time,Open,High,Low,Close,Volume
169283,2020-07-22 21:30:00,1.15686,1.15755,1.15673,1.15709,887
169284,2020-07-22 22:00:00,1.15726,1.15758,1.15714,1.15737,1338
169285,2020-07-22 22:30:00,1.15738,1.15749,1.15693,1.15697,1071
169286,2020-07-22 23:00:00,1.15697,1.15723,1.15676,1.15684,2008
169287,2020-07-22 23:30:00,1.15686,1.15698,1.15642,1.15646,1984


### Test Market
  Look at TAlib Awesome Oscillator and Zigzag indicator
  
  <ins>Zigzag indicator</ins> set to (depth 12, deviation 5, backstep 5).

In [157]:
#AO calculates the difference of a 34 Period and 5 Period Simple Moving Averages.
awesome_indicator = AwesomeOscillatorIndicator(dataM30['High'], dataM30['Low'], 5, 34)
AO = awesome_indicator.ao()

In [158]:
mid = ((dataM30['High']+dataM30['Low'])/2).to_numpy()
#0.03 and -0.03 are the minimum threshold required to define a peak
#20 pips
thresh = 20*0.0001
pivots_ = peak_valley_pivots(mid, thresh, -thresh)

In [159]:
@interact
def plot_pivots(past_n=100000, datapoints=1000):
    past_n_n = past_n-datapoints
    X = mid[-past_n:-past_n_n]
    pivots = pivots_[-past_n:-past_n_n]
    plt.xlim(0, len(X))
    plt.ylim(X.min()*0.99, X.max()*1.01)
    plt.plot(np.arange(len(X)), X, 'k:', alpha=0.5)
    
    plt.plot(np.arange(len(X))[pivots != 0], X[pivots != 0], 'k-')
    print(X[pivots != 0])
    print(len(X[pivots != 0]))
    plt.scatter(np.arange(len(X))[pivots == 1], X[pivots == 1], color='g')
    plt.scatter(np.arange(len(X))[pivots == -1], X[pivots == -1], color='r')

interactive(children=(IntSlider(value=100000, description='past_n', max=300000, min=-100000), IntSlider(value=…

In [160]:
@interact
def interactive_plot(column='Close', past_n=100000, datapoints=1000):
    #1 day == 48 intervals
    past_n_n = past_n-datapoints
    AO[-past_n:-past_n_n].plot(figsize=(10, 6));
    #return dataM30[column][-past_n:-past_n_n]

interactive(children=(Text(value='Close', description='column'), IntSlider(value=100000, description='past_n',…

### Labelling Based on rules
  Triggered only when the <ins>Awesome Oscillator</ins> arrives at a zero-crossing point. <ins>Zigzag indicator</ins> set to (depth 12, deviation 5, backstep 5).
  
  Buy = (current_bar > previous_zizag_pivot_point) and (will_increase_by_15_pips)
  
  Sell = (current_bar < previous_zizag_pivot_point) and (will_decrease_by_15_pips)
  
  ---

In [161]:
dataM30['c_label'] = ''
#if the previous values is Buy, Sell, Stationary (1, 2, 0)
labels = np.zeros([dataM30.shape[0], 3])
past_n = 15
future_n = 15
recent_n = 3
pip = 0.0001*15


for i in range(past_n, dataM30.shape[0]-future_n):
    Buy_p =  False
    Sell_p = False
    Buy =  False
    Sell = False
    
    #calculate current_std and last_3_bars
    #current_std = dataM30['Close'][i-past_n:i-recent_n].std()
    #last_3_bars = dataM30['Close'][i-recent_n:i].std()
    #last_15_close = dataM30['Close'][i-past_n:i].mean()
    
    #if zero crossing
    if (AO[i-1]*AO[i] < 0):
        #get last pivot point
        for j in range(i, 0, -1):
            if (pivots_[j] == -1):
                Buy_p = True
                break
            elif (pivots_[j] == 1):
                Sell_p = True
                break
            
        #get price delta over the next 15 bars
        for j in range(i, i+future_n):
            std_delta = dataM30['Close'][j] - dataM30['Close'][i]
            #price_delta = last_15_close - dataM30['Close'][i]
            #BUY - if up 15 bars
            if (std_delta >= pip):
            #if (std_delta >= pip and price_delta > 0):
                Buy = True
                break
            #SELL - if down 15 bars
            elif (std_delta <= -pip):
            #elif (std_delta <= -pip and price_delta > 0):
                Sell = True
                
    #BUY
    if (Buy and Buy_p):
        dataM30['c_label'][i] = 1
        labels[i][0] = 1
    #SELL
    elif (Sell and Sell_p):
        dataM30['c_label'][i] = 2
        labels[i][1] = 1
    #non-volatiles
    else:
        dataM30['c_label'][i] = 0 
        labels[i][2] = 1

    if i % 10000 == 0:
        print(i, "/", dataM30.shape[0])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


10000 / 169288
20000 / 169288
30000 / 169288
40000 / 169288
50000 / 169288
60000 / 169288
70000 / 169288
80000 / 169288
90000 / 169288
100000 / 169288
110000 / 169288
120000 / 169288
130000 / 169288
140000 / 169288
150000 / 169288
160000 / 169288


In [162]:
print("Buy signal: %d / %d" % (np.sum((dataM30['c_label'] == 1)), dataM30.shape[0]))
print("Sell signal: %d/%d" % (np.sum((dataM30['c_label'] == 2)), dataM30.shape[0]))
print("No signal: %d/%d" % (np.sum(dataM30['c_label'] == 0), dataM30.shape[0]))

Buy signal: 2470 / 169288
Sell signal: 2212/169288
No signal: 164576/169288


Buy signal: 6604 / 169288
Sell signal: 6847/169288
No signal: 155807/169288


In [32]:
#number of signals when we add an extra rule

Buy signal: 3365 / 169288
Sell signal: 3402/169288
No signal: 162491/169288


In [163]:
outputpath="./AO_EURUSD_M30.csv"
dataM30.to_csv(outputpath,sep=',',index=False,header=True)

### Buy Sell  Visualisation

In [164]:
#load csv
df_EURUSDM30 = pd.read_csv('AO_EURUSD_M30.csv')
with pd.option_context('display.precision', 20):  
    #dataM1 = pd.DataFrame(df_M1)

    dataM30=pd.DataFrame(df_EURUSDM30)
    # show all the columns
    pd.set_option('display.max_columns', None)
    # show all the rows
    pd.set_option('display.max_rows', None)

In [165]:
data = dataM30
print(list(data))
print(data.head())

['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'c_label']
                  Time     Open     High      Low    Close  Volume  c_label
0  2007-01-01 00:00:00  1.31908  1.31964  1.31896  1.31913   24827      NaN
1  2007-01-01 00:30:00  1.31920  1.31943  1.31894  1.31915   22240      NaN
2  2007-01-01 01:00:00  1.31925  1.31934  1.31882  1.31900   24568      NaN
3  2007-01-01 01:30:00  1.31910  1.31916  1.31872  1.31894   24496      NaN
4  2007-01-01 02:00:00  1.31892  1.31973  1.31874  1.31948   25233      NaN


In [166]:
data.index = pd.DatetimeIndex(dataM30.Time)
data.index.name = 'Date_Time'
data = data.drop(columns=['Time'])
#data = data.dropna()
print(data.head())

                        Open     High      Low    Close  Volume  c_label
Date_Time                                                               
2007-01-01 00:00:00  1.31908  1.31964  1.31896  1.31913   24827      NaN
2007-01-01 00:30:00  1.31920  1.31943  1.31894  1.31915   22240      NaN
2007-01-01 01:00:00  1.31925  1.31934  1.31882  1.31900   24568      NaN
2007-01-01 01:30:00  1.31910  1.31916  1.31872  1.31894   24496      NaN
2007-01-01 02:00:00  1.31892  1.31973  1.31874  1.31948   25233      NaN


In [167]:
def buy_sell(data, close_data):
    i = 0
    buy, sell, hold = [], [], []
    buy_signal, sell_signal = [], []
    for date, value in data.iteritems():
        if (value == 1):
            #buy.append((date, (close_data[date]-1)))
            buy_signal.append((close_data[date]*0.999))
            sell_signal.append(np.nan)
        elif (value == 2):
            #sell.append((date, (close_data[date]+1)))
            sell_signal.append(close_data[date]*1.001)
            buy_signal.append(np.nan)
        else:
            buy_signal.append(np.nan)
            sell_signal.append(np.nan)
    return buy_signal, sell_signal
b, s = buy_sell(data['c_label'][-200:], data['Close'][-200:])

In [168]:
mm = mid[-len(data):]
pp = pivots_[-len(data):]
AA, AA1, AA2 = np.empty(len(data)), np.empty(len(data)), np.empty(len(data))
AA[:], AA1[:], AA2[:] = np.NaN, np.NaN, np.NaN
idx = np.where(pp != 0)[0]
idx1 = np.where(pp == 1)[0]
idx2 = np.where(pp == -1)[0]


AA[idx] = mm[idx]
AA1[idx1] = mm[idx1]
AA2[idx2] = mm[idx2]
zero = np.zeros_like(AA2)

In [171]:
#interpolate zigzag pivot points to create a line
zigzag = pd.DataFrame(AA).interpolate(method ='linear', limit_direction ='forward').interpolate(method ='linear', limit_direction ='backward')

In [172]:
@interact
def interactive_plot(column='Close', past_n=100000, datapoints=1000):
    #1 day == 48 intervals
    #past_n_n = past_n-(days*48)
    past_n_n = past_n-datapoints
    
    b, s = buy_sell(data['c_label'][-past_n:-past_n_n], data['Close'][-past_n:-past_n_n])
    
    buy = mpf.make_addplot(b, type='scatter', marker='^', markersize=100, color='blue')
    sell = mpf.make_addplot(s, type='scatter', marker='v', markersize=100, color='orange')
    
    zigzag_p = mpf.make_addplot(zigzag[-past_n:-past_n_n], type='line', color='black')
    aaa1, aaa2 = [], []
    aaa1 = mpf.make_addplot(AA1[-past_n:-past_n_n], type='scatter', marker='o', markersize=50, color='green')
    aaa2 = mpf.make_addplot(AA2[-past_n:-past_n_n], type='scatter', marker='o', markersize=50, color='red')
    oscillator = mpf.make_addplot((AO[-past_n:-past_n_n]),panel=1,color='blue')
    zero_line =  mpf.make_addplot((zero[-past_n:-past_n_n]),panel=1,color='black')

    apds = []
    if (pd.notna(b).any() and pd.notna(s).any()):
        apds = [buy, sell, zigzag_p, aaa1, aaa2, oscillator, zero_line]
    elif(pd.notna(b).any()):
        apds = [buy, zigzag_p, aaa1, aaa2, oscillator, zero_line]
    elif(pd.notna(s).any()):
        apds = [sell, zigzag_p, aaa1, aaa2, oscillator, zero_line]
    
    
    mpf.plot(data[-past_n:-past_n_n], figsize=(20,10), type='candle', style='charles', title='', ylabel='', ylabel_lower='', addplot=apds)

interactive(children=(Text(value='Close', description='column'), IntSlider(value=100000, description='past_n',…